In [2]:
import requests
import time
import json
import threading
import queue
import random
import datetime
import os
import urllib3
import pickle
import re

from urllib.error import URLError, HTTPError
from bs4 import BeautifulSoup
from collections import defaultdict
from pprint import pprint
from config.assistant import *
from config.site_config import *
from tqdm import tqdm

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [9]:
# searchType=search
# searchword=컴퓨터비전         # 검색어
# loc_mcd=101000,102000         # 지역, 101000(서울),102000(경기)
# company_cd=0,1,2,3,4,5,6,7    # 채용제공구분 / 9(파견/대행), 10(헤드헌팅)
# search_optional_item=y
# search_done=y
# panel_count=y
# recruitPage=1                 # 현재 페이지뷰
# recruitSort=relation          # 정렬방법
# recruitPageCount=100          # 표시 개수
# inner_com_type=
# show_applied=                 # 지원한 공고 표시
# quick_apply=                  # 빠른 입사지원
# except_read=                  # 조회한 페이지 제외
# ai_head_hunting=
# mainSearch=n

In [2]:
search_words = ["컴퓨터비전", 'COMPUTER VISION',
                "영상처리", "IMAGE PROCESSING",
                "딥러닝", "DEEP LEARNING", 
                "머신러닝", "MACHINE LEARNING",
                "이미지인식", "IMAGE RECOGNITION",
                "이미지분석", "IMAGE ANALYSIS",
                "VISION AI",
                "OPENCV"
               ]

search_words = ["이미지인식"]
page_view_items = 100
company_cd='0,1,2,3,4,5,6,7'
base_link = 'https://www.saramin.co.kr'
job_group = {}


In [2]:
len(job_group)

NameError: name 'job_group' is not defined

In [12]:
# 머신러닝: 1450
# MACHINE LEARNING: 1445
# 딥러닝: 1158
# DEEP LEARNING: 1158
# 컴퓨터비전: 1120
# 영상처리: 1037
# COMPUTER VISION: 964
# 이미지분석: 714
# VISION AI: 657
# IMAGE PROCESSING: 295
# OPENCV: 232
# 이미지인식: 222
# IMAGE ANALYSIS: 21
# IMAGE RECOGNITION: 12

# 9365
# job_group 2259

In [13]:
# JSON 파일 쓰기
file_path = "config/job_group.json"

tf = open(file_path, 'w')
json.dump(job_group, tf)
tf.close()

In [2]:
# JSON 파일 불러오기
file_path = "config/job_group.json"
with open(file_path, 'r') as file:
    job_group = json.load(file)

In [3]:
print(len(job_group))

24


In [4]:
copy_group = {}

In [5]:
input_status = 5
if input_status == 1:      status = 'all'
elif input_status == 2:    status = 'wait'
elif input_status == 3:    status = 'save'
elif input_status == 4:    status = 'hold'
elif input_status == 5:    status = 'close'
        
# ALL   : 1
# WAIT  : 2
# SAVE  : 3
# HOLD  : 4
# CLOSE : 5

def filter_status_func():
    try:
        copy_group[companys].append(job_group[companys][idx])
    except:
        copy_group[companys] = []
        copy_group[companys].append(job_group[companys][idx])
    
for companys in job_group:
    for idx, job_list in enumerate(job_group[companys]):
        if job_group[companys][idx]['status'] == status:    # ALL
            test_func()
        elif job_group[companys][idx]['status'] == status:    # WAIT
            test_func()
        elif job_group[companys][idx]['status'] == status:    # SAVE
            test_func()
        elif job_group[companys][idx]['status'] == status:    # HOLD
            test_func()
        elif job_group[companys][idx]['status'] == status:    # CLOSE
            test_func()   

In [6]:
pprint(copy_group)

{'스텔란티스코리아': [{'company': '스텔란티스코리아',
               'company_link': 'https://www.saramin.co.kr/zf_user/company-info/view?csn=QWZyVlFHKy9RQkNRQmhSVXRDWnJ5UT09',
               'input_date': '2022-08-28',
               'status': 'close',
               'title': '[수입차 Jeep/Peugeot] Product Planning (대리-과장급)',
               'title_idx': 1,
               'title_link': 'https://www.saramin.co.kr/zf_user/jobs/relay/view?view_type=search&rec_idx=43852302&location=ts&searchword=IMAGE+ANALYSIS&searchType=search&paid_fl=n&search_uuid=1fea4ee1-80dd-49cb-a985-e7bbcf9629ff'}],
 '제이엘케이': [{'company': '제이엘케이',
            'company_link': 'https://www.saramin.co.kr/zf_user/company-info/view?csn=dGJHczlpN0NvYWNkaEhUbGRXZXdKUT09',
            'input_date': '2022-08-28',
            'status': 'close',
            'title': '[JLK] 인공지능(AI)·빅데이터연구원 채용(병역특례가능)',
            'title_idx': 0,
            'title_link': 'https://www.saramin.co.kr/zf_user/jobs/relay/view?view_type=search&rec_idx=43622025&locati

In [3]:
# JSON 파일 불러오기
file_path = "config/job_group.json"
with open(file_path, 'r') as file:
    job_group = json.load(file)
print(len(job_group))

5688


In [4]:
len(job_group['메디픽셀'])

5

In [6]:
for i in job_group['메디픽셀']:
    print(i['title'])

[전문연구요원] Computer Vision 의료 리서치 엔지니어 채용
의료영상 리서치 엔지니어 (전문연구요원 가능)
[전문연구요원] 딥러닝/영상처리/의료 엔지니어 채용
[메디픽셀] 의료기기 품질관리자(QA) 경력직 채용
Machine Learning Team Lead 채용


In [ ]:

for search_word in search_words:

    item_count = 0
    current_page = 1
    
    # 검색 결과 리스트 페이지 별 확인
    search_loop = True
    while(search_loop):
        search_link = f'https://www.saramin.co.kr/zf_user/search/recruit'\
            + f'?search_area=main'\
            + f'&search_done=y'\
            + f'&search_optional_item=n'\
            + f'&searchType=search'\
            + f'&recruitSort=relation'\
            + f'&searchword={search_word}'\
            + f'&recruitPage={current_page}'\
            + f'&recruitPageCount={page_view_items}'\
            + f'&company_cd={company_cd}'\
            + f'&mainSearch=y'
        print(search_link)
        response = requests.get(search_link, verify=False)
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')

        if len(soup.find_all('div', attrs={'class': 'info_no_result'})) == 1:
            search_loop = False
        else:
            current_page += 1

            page_items = len(soup.find_all('h2', attrs={'class': 'job_tit'}))

            # 해당 페이지 리스트 (1 ~ 100)
            for i in range(1, page_items+1):
                elements = soup.select(f'div.content > div:nth-child({i})')[0]

                # 회사명
                job_company = elements.find('a', attrs={'class': 'track_event data_layer'}).text
                filtering_result = filtering_company(job_company)
                if filtering_result[0] == 0:
                    continue
                else:
                    job_company = filtering_result[1]
                    
                # 채용공고명
                job_posting = elements.find('a', attrs={'class': 'data_layer'})['title']

                # 채용공고 링크
                posting_link = base_link_saram + elements.find('a', attrs={'class': 'data_layer'})['href']

                # 회사 링크
                company_link = base_link_saram + elements.find('a', attrs={'class': 'track_event data_layer'})['href']

                # Dictionary에 저장
                save_dictionary(job_company, company_link, job_posting, posting_link)
                
                item_count += 1
        

In [1]:
search_link = "https://www.saramin.co.kr/zf_user/search/recruit?search_area=main&search_done=y&search_optional_item=n&searchType=search&recruitSort=relation&searchword=%EC%BB%B4%ED%93%A8%ED%84%B0%EB%B9%84%EC%A0%84&recruitPage=1&recruitPageCount=100&company_cd=0,1,2,3,4,5,6,7&mainSearch=y"


In [2]:
import requests

request_headers = {
    'User-Agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                   '(KHTML, like Gecko) Chrome/68.0.3440.75 Safari/537.36'),
}

url = 'https://search.naver.com/search.naver'

response = requests.get(search_link, headers = request_headers)
print(response)

<Response [200]>


In [3]:
import requests
response = requests.get(search_link)
print(response)

<Response [400]>


In [3]:
response = requests.get(search_link, verify=False)
html = response.text
soup = BeautifulSoup(html, 'html.parser')

In [6]:
response

<Response [400]>

In [4]:
html

'<meta http-equiv="refresh" content="0;url=https://www.saramin.co.kr/error/HTTP_BAD_REQUEST.php">'

In [5]:
soup

<meta content="0;url=https://www.saramin.co.kr/error/HTTP_BAD_REQUEST.php" http-equiv="refresh"/>

In [6]:
soup.find_all('h2', attrs={'class': 'job_tit'})

[]

In [ ]:
#recruit_info > div.header > span

In [ ]:
<span class="cnt_result">총 1,246건</span>